In [77]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [78]:
torch.manual_seed(42)

In [79]:
df=pd.read_csv('./datasets/fashion-mnist_train.csv')
df.head(1)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [81]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [82]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [83]:
x_train=x_train/225.0
x_test=x_test/225.0

In [84]:
class customDataset(Dataset):
    def __init__(self,features,labels):
        self.features=torch.tensor(features,dtype=torch.float32)
        self.labels=torch.tensor(labels,dtype=torch.long)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,ind):
        return self.features[ind],self.labels[ind]
    

In [85]:
train_dataset = customDataset(x_train,y_train)
test_dataset = customDataset(x_test,y_test)

In [86]:
test_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [87]:
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_dataloader= DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [88]:
class SimpleANN(nn.Module):
    def __init__(self,num_of_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_of_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)         
        )
    def forward(self,features):
        return self.model(features)

In [89]:
epochs=100
lr=0.1

In [90]:
model=SimpleANN(x_train.shape[1]).to(device)

In [91]:
citeration = nn.CrossEntropyLoss()

In [92]:
optimizer = optim.SGD(model.parameters(),lr=lr)

In [93]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features,batch_labels in train_dataloader:
        batch_features,batch_labels = batch_features.to(device),batch_labels.to(device)
        pred=model(batch_features)
        loss = citeration(pred,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss+=loss.item()
    avg_loss = total_epoch_loss/len(train_dataloader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')
        

Epoch: 1 , Loss: 0.6245252492825191
Epoch: 2 , Loss: 0.42743367994825043
Epoch: 3 , Loss: 0.38411147879064084
Epoch: 4 , Loss: 0.35705055251717566
Epoch: 5 , Loss: 0.3344689627836148
Epoch: 6 , Loss: 0.3218810750817259
Epoch: 7 , Loss: 0.30614484356095395
Epoch: 8 , Loss: 0.2952117714360356
Epoch: 9 , Loss: 0.28427954148997864
Epoch: 10 , Loss: 0.2734819379573067
Epoch: 11 , Loss: 0.26775027192632356
Epoch: 12 , Loss: 0.2581611140432457
Epoch: 13 , Loss: 0.24949316652740042
Epoch: 14 , Loss: 0.2428294148805241
Epoch: 15 , Loss: 0.2378063898210724
Epoch: 16 , Loss: 0.23197789440179864
Epoch: 17 , Loss: 0.22369262045497695
Epoch: 18 , Loss: 0.22065019097054997
Epoch: 19 , Loss: 0.2151844173396627
Epoch: 20 , Loss: 0.21033578078572948
Epoch: 21 , Loss: 0.2052320196852088
Epoch: 22 , Loss: 0.19943672912940383
Epoch: 23 , Loss: 0.194382314397643
Epoch: 24 , Loss: 0.19405937441190083
Epoch: 25 , Loss: 0.1878156934340174
Epoch: 26 , Loss: 0.18645072725228964
Epoch: 27 , Loss: 0.18046945264873

In [94]:
model.eval()

SimpleANN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [95]:
correct=0
total=0
with torch.no_grad():
    for batch_features,batch_labels in test_dataloader : 
        batch_features,batch_labels= batch_features.to(device),batch_labels.to(device)
        pred= model(batch_features)
        _,predicted=torch.max(pred,1)
        total+=batch_labels.shape[0]
        correct+=(predicted==batch_labels).sum().item()
    print(correct/total)
    

0.8829166666666667
